In [ ]:
from pynq import Overlay
from pynq import MMIO
import numpy as np
import time


In [ ]:
ifmap = np.load('input.npy')

In [ ]:
class PynqTestDriver:
    def __init__(self, bitfile_path):
        self.hw = Overlay(bitfile_path)
        self.csr = self.hw.csr_0.mmio.array
        self.imem = self.hw.SRAM1.mmio.array
        self.omem = self.hw.SRAM2.mmio.array

    def start(self, input_array):
        start_time = time.time()
        self.imem[0:10404] = input_array
        self.csr[1] = 1
        while True:
            if (self.csr[0] == 1):
                break
        self.result = self.omem[0:10000]
        end_time = time.time()
        runtime = end_time - start_time
        print(f"Runtime: {runtime*1000:.3f}ms")

In [11]:
hw = PynqTestDriver('bd_conv_core.bit')

In [12]:
hw.start(input_data)

Runtime: 4.650ms


In [13]:
hw.generate_img("result_img.jpg")

In [14]:
result = hw.get_output(10000)

In [15]:
# this command generate solution for sobel edge detector

from scipy.signal import convolve2d

sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]], dtype=np.float32)

sobel_y = np.array([[-1,  -2,  -1],
                    [ 0,  0,  0],
                   [1,2,1]], dtype=np.float32)

grad_x = convolve2d(image, sobel_x, mode='valid')
grad_y = convolve2d(image, sobel_y, mode='valid')
abs_sobel_x = np.abs(grad_x)
abs_sobel_y = np.abs(grad_y)

sobel = abs_sobel_x + abs_sobel_y

grad_clipped = np.clip(sobel, 0, 255).astype(np.uint8)
print(grad_clipped)
cv2.imwrite('answer_image.png', grad_clipped)


[[ 72  78  80 ...   4   0   6]
 [ 66  68  66 ...   6   6   8]
 [ 24  30  34 ...   6   6  10]
 ...
 [ 76  72  72 ... 114 148 100]
 [ 26  12  12 ...  62 136 202]
 [ 48  34  38 ... 104  36  68]]


True

In [16]:
hw.compare_result(grad_clipped)

END


In [17]:
# this command generate hex file for verilog simulation

def image_to_mem(image_path, mem_path):

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img.shape != (102, 102):
        img = cv2.resize(img,(102,102))

    flat = img.flatten()

    hex_vals = [f"{val:02X}" for val in flat]

    with open(mem_path, 'w') as f:
        for val in hex_vals:
            f.write(val + '\n')

    print(f"[✓] Done for making Hex file for readmemh: {mem_path}")


image_to_mem("Penguin.jpg", "output_image.hex")

[✓] Done for making Hex file for readmemh: output_image.hex
